In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
from fastai.vision.all import *

labels = pd.read_csv("../input/dog-breed-identification/labels.csv")
labels


In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_ids, valid_ids = next(split.split(labels, labels["breed"]))
labels["is_valid"] = [i in valid_ids for i in range(len(labels))]

labels["id"] = labels["id"].apply(lambda x: x + ".jpg")


In [ ]:
len(dls.vocab)


In [ ]:
path = "../input/dog-breed-identification/train"

dls = ImageDataLoaders.from_df(labels, path,
                               item_tfms=Resize(460, method="squeeze"),
                               batch_tfms=[*aug_transforms(size=300),
                                           Normalize.from_stats(*imagenet_stats)],
                               bs=32, valid_col="is_valid")
dls.show_batch()


In [ ]:
import torchvision.models as models

inception = models.inception_v3(pretrained=True)
inception.fc = nn.Linear(2048, 200)
inception = inception.eval()


In [ ]:
resnet = models.resnet50(pretrained=True)
resnet.fc = nn.Linear(2048, 200)
resnet = resnet.eval()


In [ ]:
class NeuralNet(Module):
    def __init__(self, extractors, device="cpu"):
        self.extractors = extractors
        for conv in self.extractors:
            conv.to(device)
        self.classifier = nn.Linear(400, len(dls.vocab)).to(device)
        
    def forward(self, x):
        
        features = [conv(x) for conv in self.extractors]
        features = torch.cat(features, dim=1)
        
        return self.classifier(features)


In [ ]:
extractors = [inception, resnet]
device = "cuda" if torch.cuda.is_available() else "cpu"
model = NeuralNet(extractors, device)


In [ ]:
learn = Learner(dls, model, metrics=accuracy, path=".").to_fp16()
learn.lr_find()


In [ ]:
learn.fit_one_cycle(5, 1e-2)


In [ ]:
test_files = get_image_files("../input/dog-breed-identification/test")
test_dl = dls.test_dl(test_files, bs=16)


In [ ]:
preds, targs = learn.get_preds(dl=test_dl)


In [ ]:
#preds = torch.softmax(preds, dim=1)
sub = pd.DataFrame({"id":test_files.map(lambda x:x.stem)})
sub[list(dls.vocab)] = preds
sub.to_csv("submission.csv", index=False)


In [ ]:
sub
